<a href="https://colab.research.google.com/github/feliciakiani/PolitikPedia/blob/main/Machine%20Learning/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data preprocessing

In [2]:
!pip install PySastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 4.3 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving new_kamusalay.csv to new_kamusalay.csv


In [6]:
import nltk
from nltk.corpus import stopwords

# Download the NLTK stop words dataset (if not already downloaded)
nltk.download('stopwords')

# Get the list of Indonesian stop words
stop_words_indonesian = set(stopwords.words('indonesian'))
id_stopword_dict = pd.DataFrame({'stop_word': list(stop_words_indonesian)})

alay_dict = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original',
                                      1: 'replacement'})

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()


def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    # Remove every '\n'
    text = re.sub(r'\n', ' ', text)

    # Remove every retweet symbol
    text = re.sub(r'rt', ' ', text)

    # Remove every username
    text = re.sub(r'user', ' ', text)

    # Remove every URL
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', ' ', text)

    # Remove all emojis (Unicode characters)
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]', ' ', text)

   # Remove all hexadecimal representations of UTF-8 encoded characters
    text = re.sub(r'\\x[0-9a-fA-F]{2}', ' ', text)
    text = re.sub(r'x[0-9a-fA-F]{2}', ' ', text)  # If the 'x' is not escaped

    # Remove extra spaces
    text = re.sub(r'  +', ' ', text)

    return text

def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    return text

alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

def remove_stopword(text):
    stop_words = set(id_stopword_dict['stop_word'])
    text = ' '.join(['' if word in stop_words else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text)  # Remove extra spaces
    text = text.strip()
    return text

def stemming(text):
    return stemmer.stem(text)

def preprocess(text):
    text = lowercase(text) # 1
    text = remove_nonaplhanumeric(text) # 2
    text = remove_unnecessary_char(text) # 2
    text = normalize_alay(text) # 3
    text = stemming(text) # 4
    text = remove_stopword(text) # 5
    return text

In [ ]:
!pip install mysql-connector-python
import mysql.connector
from textblob import TextBlob
from keras.preprocessing.sequence import pad_sequences

#kodingannya taru diamana ? access modelnya taro dimana gitu

# Koneksi ke database Cloud SQL
conn = mysql.connector.connect(
    host='your-cloud-sql-host',
    user='your-username',
    password='your-password',
    database='your-database'
)

# Buat kursor
cursor = conn.cursor()

# Query data komentar
query = "SELECT id, comment_text FROM comments"
cursor.execute(query)
comments = cursor.fetchall()

#-------------------------------------------------Text_Classification-------------------------------------------#
#Hyperparameter, jangan diganti
max_length = 56

# Loop through each comment, gausah diganti
for comment_id, comment_text in comments:
    # Apply preprocessing
    preprocessed_text = preprocess(comment_text)
    # Tokenize and pad the new text
    new_sequence = tokenizer.texts_to_sequences([comment_text])
    new_padded_sequence = pad_sequences(new_sequence, maxlen=max_length, padding='post')

    # Make predictions
    prediction = model.predict(new_padded_sequence)

    # Extract the scalar value from the NumPy array
    confidence = prediction[0, 0]

    # Threshold for considering a label as positive
    threshold = 0.5

    # Interpret prediction
    predicted_class = 1 if confidence >= threshold else 0

    # Convert confidence to a format that can be handled by format method
    confidence_str = '{:.4f}'.format(confidence)

    print(f"Comment ID: {comment_id}, Predicted Class: {predicted_class} (Confidence: {confidence_str})")
#-------------------------------------------------Text_classification-------------------------------------------#
# Tutup koneksi
cursor.close()
conn.close()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 37.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


DatabaseError: ignored